# Ключевые слова и tf-idf

На прошлом занятии мы учились находить в тексте коллокации, сравнивая вероятности появления в тексте биграммы и слов, входящих в эту биграмму, по отдельности. Но если мы хотим оценить содержание текста, не читая его, и рассклассифицировать тексты по темам, одних коллокаций мало.

О содержании текста много рассказывают ключевые слова. Что такое ключевое слово? Коллокации мы находили в одном-единственном тексте, а ключевые слова в одном отдельно взятом тексте не существуют. С точки зрения компьютерной лингвистики ключевые слова - это слова, которые часто встречаются в этом документе и редко во всех остальных. Если мы возьмём абстрактный текст про Францию, ключевыми словами будут сама _Франция_ и _Париж_, потому что они встретятся в этом тексте, а в текстах про другие страны или вообще не про страны _Франция_ и _Париж_ встречаются реже. 

Однако все-таки встречаются, правда? В тексте про знаменитые музеи мира, возможно, мы тоже встретим слова _Париж_ и _Франция_. когда речь зайдёт о Лувре, однако нельзя сказать, чтобы они были ключевыми. В чём разница между текстом про Францию и текстом про Лувр? Одно из различий - что в описании Лувра Париж будет упомянут только один раз, а в статье про Францию, возможно, и больше одного раза.

__Тогда из чего складывается понятие "ключевое слово"?__

* если слово появляется в документе часто, значит, оно важное и может быть ключевым.
* если это же слово часто встречается и в других документах, значит, оно не такое же важное и ключевое.

Ключевые слова должны быть частотными в конкретном документе и редкими во всех остальных. Для того, чтобы оценить степень важности слова в этом тексте, существует метрика, которая называется [__tf-idf (term frequency - inverted document frequency)__](https://en.wikipedia.org/wiki/Tf%E2%80%93idf).

__term__ - это слово в тексте (более-менее)
__document__ - это отдельный текстовый документ, будь то статья в Википедии, или новость, или анекдот.

В метрике tf-idf учитываются те два компонента "ключевости" слова, о которых мы говорили раньше: 

* term frequency - насколько часто слово встречается в документе.
* inverted document frequency - насколько редко слово встречается во всех остальных документах.

Сама метрика представляет собой просто произведение этих двух частот.

Как вы думаете, с какими текстами tf-idf работает хорошо, а с какими плохо? Будет ли tf-idf хорошо работать с очень короткими текстами (типа твитов)? А с очень длинными (типа романов Льва Толстого)?

Давайте посчитаем tf-idf для слов в 100 статьях из русской Википедии.

In [1]:
def count_tf(word, text): # считаем, сколько раз слово встретилось в тексте и делим на общее количество слов (как в pmi)
    return text.count(word) / len(text)

In [2]:
def count_df(word, texts): # считаем, в скольких текстах встретилось слово
    n = [1 for text in texts if word in text]
    return sum(n)

In [3]:
# но нам нужна не прямая частота, а обратная
def count_idf(word, texts):
    n = len(texts) / (1 + count_df(word, texts)) # Зачем здесь единица?
    return n

In [4]:
# наконец считаем сам tf-idf. Как и во многих случаях, хорошая практика - брать не сами числа, а их логарифмы
from math import log
def count_tfidf(word, text, texts):
    tf = count_tf(word, text)
    idf = count_idf(word, texts)
    return log(tf, 10) * log(idf, 10)

Все, теперь мы можем обрабатывать тексты. Однако сначала их надо предобработать:

In [5]:
import re

punct = '[.,!«»?&@"$\[\]\(\):;%#&\'—-]'

def preprocessing(text): # функция предобработки текста
    text_wo_punct = re.sub(punct, '', text.lower()) # удаляем пунктуацию, приводим в нижний регистр
    words = text_wo_punct.strip().split() # делим по пробелам
    return words

In [6]:
import os

texts_dic = {}
for root, dirs, files in os.walk('wikipedia'):
    for f in files[:50]:
        with open(os.path.join(root, f), 'r', encoding='utf-8') as t:
            text = preprocessing(t.read())
            texts_dic[f.split('.')[0]] = text
texts = list(texts_dic.values())

In [8]:
for text in texts_dic:
    print("Top words in document {}".format(text)) # чтобы было понятно, какой документ мы обрабатывали
    scores = {} # здесь будем хранить оценки для каждого текста
    for word in texts_dic[text]:
        scores[word] =  count_tfidf(word, texts_dic[text], texts)
    sorted_words = sorted(scores.items(), key=lambda x: x[1])
    for word, score in sorted_words[:5]:
        print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))

Top words in document Эскадренные миноносцы типов Z и Ca
	Word: mark, TF-IDF: -3.9876
	Word: пароперегревателями, TF-IDF: -3.9876
	Word: строились, TF-IDF: -3.9876
	Word: спаренный, TF-IDF: -3.9876
	Word: склонения, TF-IDF: -3.9876
Top words in document Саботаж! (фильм, 2000)
	Word: плену, TF-IDF: -3.65694
	Word: рейтинг, TF-IDF: -3.65694
	Word: духа, TF-IDF: -3.65694
	Word: временами, TF-IDF: -3.65694
	Word: подлинного, TF-IDF: -3.65694
Top words in document Никита (Латушко)
	Word: всеволожска, TF-IDF: -3.45677
	Word: всеволожске, TF-IDF: -3.45677
	Word: вывезено, TF-IDF: -3.45677
	Word: белорусская, TF-IDF: -3.45677
	Word: наместничество, TF-IDF: -3.45677
Top words in document Поход уральцев из форта Александровского в Персию
	Word: карамышев, TF-IDF: -4.29225
	Word: константинополь, TF-IDF: -4.29225
	Word: архистратига, TF-IDF: -4.29225
	Word: месяца, TF-IDF: -4.29225
	Word: даль­ше, TF-IDF: -4.29225
Top words in document Midori Days
	Word: тело, TF-IDF: -4.22344
	Word: такаги, TF-I

# Задания

1. Попробуйте улучшить результат, очистив тексты от служебных и коротких слов.
2. Скомбинируйте PMI и tf-idf: сначала выделите коллокации при помощи PMI, а затем отберите наиболее значимые из этих коллокаций, вычислив их tf-idf score.